In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster

In [2]:
key = open('../keys/google.txt').readline()

In [3]:
damage = pd.read_csv('../data/Mapping Home Loss.csv')
print(damage.shape)
damage.head()

(385, 8)


,Address,Name,Served by SCT?,Before photos,After photos,Story,Notes,Unnamed: 7
0,"Hickory Tree Road, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,Off Lower Grassy Branch,NaN
1,"Patton Pl, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,Between Grassy Branch and Swannanoa River,NaN
2,"Riverview Parkway, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,Directly across the river from Driftwood,NaN
3,"Sunnybrook Drive, Asheville 28805",Bill Queen,NaN,NaN,NaN,NaN,There is a trailer park east of River Knoll th...,NaN
4,"382 Old Farm School Road, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,displaced total loss,NaN


In [4]:
damage.tail()

,Address,Name,Served by SCT?,Before photos,After photos,Story,Notes,Unnamed: 7
380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
382,(DO NOT INCLUDE IN MAP),NaN,NaN,NaN,NaN,NaN,NaN,NaN
383,53 midcourt drive,April s,NaN,This is North Asheville - not swannanoa?,NaN,NaN,NaN,NaN
384,Grovemont,Alegra West,NaN,"Need more info, complete address",NaN,NaN,NaN,NaN


In [5]:
addresses = damage.Address
addresses[49:60]

49                  55 Briggs Road, Asheville, NC 28805
50                            103 old jim's branch road
51                  2 Old Jims Branch Rd, swannanoa, nc
52                                Old Jim's Branch Road
53                                                  NaN
54    213 old us highway 70 east, black mountain, NC...
55                                                  NaN
56               160 Old US Hwy 70, Swannanoa, NC 28778
57                   206 Old US 70, Swannanoa, NC 28778
58                   207 Old Us 70, Swannanoa, NC 28778
59                   212 Old US 70, Swannanoa, NC 28778
Name: Address, dtype: object

In [6]:
def get_coordinates(key, address):
    endpoint_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    params = {
        'address': address,
        'key': key
    }
    try:
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        return results
    except Exception as e:
        print('error: ', address, e) 

In [7]:
geo_addr = []
for address in addresses:
    geo_addr.append(get_coordinates(key, address))

In [8]:
print(geo_addr[0]['results'][0]['geometry']['location'])

{'lat': 35.5910613, 'lng': -82.4703527}


In [9]:
geometries = []
error_list = []
for i in range(len(geo_addr)):
    try:
        geometries.append(geo_addr[i]['results'][0]['geometry']['location'])
    except:
        error_list.append(i)
        print('cannot add' , i)

cannot add 45
cannot add 46
cannot add 47
cannot add 48
cannot add 53
cannot add 55
cannot add 92
cannot add 97
cannot add 101
cannot add 110
cannot add 130
cannot add 131
cannot add 132
cannot add 172
cannot add 174
cannot add 176
cannot add 228
cannot add 231
cannot add 258
cannot add 262
cannot add 264
cannot add 276
cannot add 280
cannot add 290
cannot add 336
cannot add 341
cannot add 344
cannot add 376
cannot add 377
cannot add 378
cannot add 379
cannot add 380
cannot add 381
cannot add 382


In [10]:
geo_addr[45]

{'results': [], 'status': 'ZERO_RESULTS'}

In [11]:
len(geo_addr)

385

In [12]:
len(error_list)

34

In [13]:
damage.loc[error_list]

,Address,Name,Served by SCT?,Before photos,After photos,Story,Notes,Unnamed: 7
45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
damage = damage.drop(error_list)

In [15]:
damage['geometry'] = [Point(geo['lng'], geo['lat']) for geo in geometries]

In [16]:
damage.head()

,Address,Name,Served by SCT?,Before photos,After photos,Story,Notes,Unnamed: 7,geometry
0,"Hickory Tree Road, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,Off Lower Grassy Branch,NaN,POINT (-82.4703527 35.5910613)
1,"Patton Pl, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,Between Grassy Branch and Swannanoa River,NaN,POINT (-82.4715476 35.5944806)
2,"Riverview Parkway, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,Directly across the river from Driftwood,NaN,POINT (-82.4642206 35.60033)
3,"Sunnybrook Drive, Asheville 28805",Bill Queen,NaN,NaN,NaN,NaN,There is a trailer park east of River Knoll th...,NaN,POINT (-82.4622483 35.6069665)
4,"382 Old Farm School Road, Asheville, NC 28805",NaN,NaN,NaN,NaN,NaN,displaced total loss,NaN,POINT (-82.4701133 35.6079282)


In [17]:
damage = damage.loc[damage.Address != '53 midcourt drive']

In [19]:
damage['lng'] = damage.geometry.apply(lambda p: p.x)
damage['lat'] = damage.geometry.apply(lambda p: p.y)

In [20]:
center = [35.6011002,-82.4151444]
helene = folium.Map(location = center, zoom_start = 12)
helene

In [21]:
for row_index, row_values in damage.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['Address'])
    if row_values['Notes'] == 'displaced total loss':
        color = 'red'
    else:
        color = 'orange'
    icon=folium.Icon(color=color,icon=" fa-exclamation-circle", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(helene)
helene.save('../maps/helene_map.html')

helene

In [22]:
area_center = [35.6011002,-82.4151444]

cluster_helene = folium.Map(location =  area_center, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_helene)


# inside the loop add each marker to the cluster
for row_index, row_values in damage.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['Address'])
    if row_values['Notes'] == 'displaced total loss':
        color = 'red'
    else:
        color = 'orange'
    icon=folium.Icon(color=color,icon=" fa-exclamation-circle", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)
cluster_helene.save('../maps/helene_cluster_map.html')

cluster_helene